In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, Imputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
import matplotlib
import matplotlib.pylab as plt
%matplotlib inline

Load the ethereum dataset

In [2]:
df = pd.read_csv('ethereum_dataset.csv')

Drop all columns but `UnixTimeStamp` & `eth_etherprice`

In [3]:
df = df[['UnixTimeStamp','eth_etherprice']].copy()

Define a function that calculates the logarithmic return (continuously compounded return / force of interest) for every record per the equation:
$$R = \ln \left({\frac {V_{f}}{V_{i}}}\right)$$
Where __Vf__ is the final value and __Vi__ is the initial value

In [4]:
def calculateLogReturn(df, numDays_windowLag, value_col, timestamp_col, result_col):
    secondsInDay = 86400
    
    # Create a copy of df for processing
    df_copy = df.copy()
    
    # Create columns for 'final value' and initial value' timestamps 
    df_copy['finalTimeStamp'] = df[timestamp_col] - ((numDays_windowLag) * secondsInDay)
    df_copy['initialTimeStamp'] = df[timestamp_col] - ((numDays_windowLag+1) * secondsInDay)
    
    # create an array to collect results
    results = []
    
    # iterate over df_copy and generate log return for each row
    for i, row in df_copy.iterrows():
        finalTimeStamp = row['finalTimeStamp']
        initialTimeStamp = row['initialTimeStamp']
        currentValSeries = df.loc[(df[timestamp_col] == finalTimeStamp), value_col]
        initialValSeries = df.loc[(df[timestamp_col] == initialTimeStamp), value_col]
        currentVal = None
        initialVal = None
        logReturn = None
        
        # Debug Statements
        #print "-------------------"
        #print "Row: " + str(i)
        
        # if there is a value for 'value_col' for initialTimeStamp  
        if not initialValSeries.empty:
            initialVal = float(initialValSeries)
            
            #print "InitialVal: " + str(initialVal)
            # if there is a value for 'value_col' for currentTimeStamp
            if not currentValSeries.empty:
                currentVal = float(currentValSeries)
                
                #print "CurrentVal: " + str(currentVal)
                # to avoid divide by zero
                if not (initialVal == 0):
                    # calculate log return
                    finalOverInitialValue = (currentVal / initialVal)
                    #print "fraction: " + str(finalOverInitialValue)
                    # If ln(0), avoid calculation and leave null
                    if(finalOverInitialValue != 0):
                        logReturn = np.log(finalOverInitialValue)
                        #print "R: " + str(logReturn)

        results.append(logReturn)
    
    # copy results into result_col
    df[result_col] = results

Calculate log returns for windows ranging from the previous day to the previous 10 days

In [5]:
calculateLogReturn(df, 0, 'eth_etherprice','UnixTimeStamp', 'logReturn')
calculateLogReturn(df, 1, 'eth_etherprice','UnixTimeStamp', 'logReturn-1')
calculateLogReturn(df, 2, 'eth_etherprice','UnixTimeStamp', 'logReturn-2')
calculateLogReturn(df, 3, 'eth_etherprice','UnixTimeStamp', 'logReturn-3')
calculateLogReturn(df, 4, 'eth_etherprice','UnixTimeStamp', 'logReturn-4')
calculateLogReturn(df, 5, 'eth_etherprice','UnixTimeStamp', 'logReturn-5')
calculateLogReturn(df, 6, 'eth_etherprice','UnixTimeStamp', 'logReturn-6')
calculateLogReturn(df, 7, 'eth_etherprice','UnixTimeStamp', 'logReturn-7')
calculateLogReturn(df, 8, 'eth_etherprice','UnixTimeStamp', 'logReturn-8')
calculateLogReturn(df, 9, 'eth_etherprice','UnixTimeStamp', 'logReturn-9')
calculateLogReturn(df, 10, 'eth_etherprice','UnixTimeStamp', 'logReturn-10')

Filter out rows that have a null value for any log-return feature

In [6]:
df = df[df['logReturn'].notnull()]
df = df[df['logReturn-1'].notnull()]
df = df[df['logReturn-2'].notnull()]
df = df[df['logReturn-3'].notnull()]
df = df[df['logReturn-4'].notnull()]
df = df[df['logReturn-5'].notnull()]
df = df[df['logReturn-6'].notnull()]
df = df[df['logReturn-7'].notnull()]
df = df[df['logReturn-8'].notnull()]
df = df[df['logReturn-9'].notnull()]
df = df[df['logReturn-10'].notnull()]

Define a function that will calculate the labels for the training set 
Function returns a boolean (1 or 0) indicating whether next day log return is positive or not
(assumes that log return with desired window has been calculated for all records)

In [7]:
def calculateNextDayReturnSign(df, logReturn_value_col, timestamp_col, result_col):
    secondsInDay = 86400
    
    # Create a copy of df for processing
    df_copy = df.copy()
    
    # Create a column for 'next day' timestamp 
    df_copy['nextDayTimeStamp'] = df[timestamp_col] + secondsInDay
    
    # create an array to collect results
    results = []
    
    # iterate over df_copy and generate log return prediction for each row
    for i, row in df_copy.iterrows():
        nextDayTimeStamp = row['nextDayTimeStamp']
        nextDaylogReturnSeries = df.loc[(df[timestamp_col] == nextDayTimeStamp), logReturn_value_col]
        nextDayVal = None
        isLogReturnPositive = None
        
        # if there is a value for 'logReturn_value_col' for the next day  
        if not nextDaylogReturnSeries.empty:
            nextDayLogReturnVal = float(nextDaylogReturnSeries)
            
            if (nextDayLogReturnVal > 0):
                isLogReturnPositive = 1
            else:
                isLogReturnPositive = 0
        
        results.append(isLogReturnPositive)
    
    # copy results into result_col
    df[result_col] = results

Calculate labels for dataset ( `nextDayLogReturnPositive` ) 

In [8]:
calculateNextDayReturnSign(df, 'logReturn', 'UnixTimeStamp', 'nextDayLogReturnPositive')

Drop last row as we cannot create a testing label for it (no next day available to build label from)

Convert labels to int

In [9]:
df = df.iloc[:-1].copy()
df.nextDayLogReturnPositive = df.nextDayLogReturnPositive.astype(int)

Drop timestamp column for predictions

In [10]:
df = df.drop(['UnixTimeStamp'], axis=1)

Split dataset into training and testing subsets

In [11]:
y = 'nextDayLogReturnPositive'
n = int(len(df)/2)
df_train, df_test = df.iloc[:n].copy(), df.iloc[n:].copy()

X_train, y_train = df_train[df_train.columns.drop(y)], df_train[y]
X_test, y_test = df_test[df_test.columns.drop(y)], df_test[y]

X_test_with_price = X_test.copy()
#X_test = X_test[X_test.columns.drop('eth_etherprice')]
#X_train = X_train[X_train.columns.drop('eth_etherprice')]

Build a pipeline to process the dataset, build a model, and predict values:

In [12]:
knn_pipeline = Pipeline([
    ('my_std_scaler', StandardScaler()),
    ('masteralg', KNeighborsClassifier())    
])

Process dataset (normalizing features), then run KNeighborsClassifier fit/predict

In [13]:
knn_pipeline.fit(X_train, y_train)
y_pred = knn_pipeline.predict(X_test)

Print Confusion Matrix for predicted Y values:

In [14]:
print sklearn.metrics.confusion_matrix(y_test, y_pred)

[[ 57 149]
 [ 49 149]]


Exract true negative (tn), false positive (fp), false negative (fn), and true positive (tp) counts

In [15]:
tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_test, y_pred).ravel()
(tn, fp, fn, tp)

(57, 149, 49, 149)

Calculate additional metrics based on confusion matrix

In [16]:
num_predicted_correctly = tn + tp
tpr = float(tp)/(tp+fp) # precision / true positive rate
tnr = float(tn)/(tn+fp) # true negative rate
ppv = float(tp)/(tp+fn) # recall / positive predictive value
npv = float(tn)/(tn+fn) # negative predictive value

In [17]:
# f1 = 2.0/(1.0/ppv+1.0/tpr)
f1 = sklearn.metrics.f1_score(y_test, y_pred)

Define a function that will take a starting amount of "cash" and simulate buying and selling of ethereum each day (per the provided dataset) depending on the provided prediction

In [18]:
def simulateEthereumPurchases(df, y_pred, price_col, start_df_index, start_USD_amount):
    curr_USD_amount = start_USD_amount
    curr_ethereum_amount = 0.0
    final_ethereum_price = float(df.iloc[-1:][price_col])
    
    # iterate over df and buy or sell all on each day depending on 
    for i, row in df.iterrows():
        #print i
        curr_ethereum_price = row['eth_etherprice']
        curr_prediction = y_pred[i-start_df_index]
        
        # If you have USD and prediction is positive
        if curr_prediction and (curr_USD_amount > 0):
            curr_ethereum_amount += (curr_USD_amount / curr_ethereum_price)
            curr_USD_amount = 0.0
            #print 'BUY: ' + str(curr_ethereum_amount) + ' ethereum at $' + str(curr_ethereum_price)
        # If you have ethereum and prediction is negative    
        elif (not curr_prediction) and (curr_ethereum_amount > 0): 
            #print 'SELL: ' + str(curr_ethereum_amount) + ' ethereum at $' + str(curr_ethereum_price)
            curr_USD_amount += (curr_ethereum_amount * curr_ethereum_price)
            curr_ethereum_amount = 0.0
        # Else hold for the day
        #else:
            #print 'HOLD - Net Value:' + str((curr_ethereum_amount * curr_ethereum_price) if (curr_ethereum_amount > 0) else curr_USD_amount)
        
    #After processing all days, return net worth in USD
    return (curr_ethereum_amount * curr_ethereum_price) if (curr_ethereum_amount > 0) else curr_USD_amount

Calculate resulting net worth if using predictions from K Nearest Neighbors to decide when to buy / sell ethereum with test dataset

In [19]:
knn_pred_net_worth = simulateEthereumPurchases(X_test_with_price, y_pred, 'eth_etherprice', 427, 1000)

Create a simple dataframe to hold and display metrics, then load the metrics for this execution

In [20]:
metrics_df = pd.DataFrame(index=np.arange(0, 3), columns=['Technique', '% Predicted Correctly', '% of True Positives', '% of True Negatives', '% of False Positives', '% of False Negatives' , 'F1 Score', 'Final Net Worth'])
metrics_df.loc[0] = ['KNN', ((float(num_predicted_correctly)/len(y_test))*100), ((float(tp)/len(y_test))*100), ((float(tn)/len(y_test))*100), ((float(fp)/len(y_test))*100), ((float(fn)/len(y_test))*100), f1, knn_pred_net_worth]
metrics_df

,Technique,% Predicted Correctly,% of True Positives,% of True Negatives,% of False Positives,% of False Negatives,F1 Score,Final Net Worth
0,KNN,50.9901,36.8812,14.1089,36.8812,12.1287,0.600806,14439.6
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Build a pipeline to process the dataset, build a model, and predict values using LogisticRegression:

In [21]:
logistic_regression_pipeline = Pipeline([
    ('my_std_scaler', StandardScaler()),
    ('masteralg', LogisticRegression())    
])

Process dataset (normalizing features), then run LogisticRegression fit/predict

In [22]:
logistic_regression_pipeline.fit(X_train, y_train)
y_pred = logistic_regression_pipeline.predict(X_test)

Print Confusion Matrix for predicted Y values:

In [23]:
print sklearn.metrics.confusion_matrix(y_test, y_pred)

[[189  17]
 [174  24]]


Exract true negative (tn), false positive (fp), false negative (fn), and true positive (tp) counts

In [24]:
tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_test, y_pred).ravel()

Create additional metrics based on confusion matrix

In [25]:
num_predicted_correctly = tn + tp
tpr = float(tp)/(tp+fp) # precision / true positive rate
tnr = float(tn)/(tn+fp) # true negative rate
ppv = float(tp)/(tp+fn) # recall / positive predictive value
npv = float(tn)/(tn+fn) # negative predictive value

In [26]:
# f1 = 2.0/(1.0/ppv+1.0/tpr)
f1 = sklearn.metrics.f1_score(y_test, y_pred)

Calculate resulting net worth if using predictions from logarithmic regression model to decide when to buy / sell ethereum with test dataset

In [27]:
log_reg_pred_net_worth = simulateEthereumPurchases(X_test_with_price, y_pred, 'eth_etherprice', 427, 1000)

Load the metrics from this execution into the metrics dataframe

In [28]:
metrics_df.loc[1] = ['Logistic Regression', ((float(num_predicted_correctly)/len(y_test))*100), ((float(tp)/len(y_test))*100), ((float(tn)/len(y_test))*100), ((float(fp)/len(y_test))*100), ((float(fn)/len(y_test))*100), f1, log_reg_pred_net_worth]
metrics_df

,Technique,% Predicted Correctly,% of True Positives,% of True Negatives,% of False Positives,% of False Negatives,F1 Score,Final Net Worth
0,KNN,50.9901,36.8812,14.1089,36.8812,12.1287,0.600806,14439.6
1,Logistic Regression,52.7228,5.94059,46.7822,4.20792,43.0693,0.200837,1124.65
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Calculate Net Worth if simply buying on first day of test dataset, and selling on last day of test dataset

In [29]:

simple_net_worth = 1000
# Divide $1000 by initial ethereum value
total_ethereum_count = (simple_net_worth / float(df_test.iloc[0]['eth_etherprice']))
# Multiply total_ethereum_count by final ethereum value
simple_net_worth = (total_ethereum_count * float(df_test.iloc[-1:]['eth_etherprice']))

metrics_df.loc[2] = ['Buy First Day, Sell Last Day', None, None, None, None, None, None, simple_net_worth]
metrics_df

,Technique,% Predicted Correctly,% of True Positives,% of True Negatives,% of False Positives,% of False Negatives,F1 Score,Final Net Worth
0,KNN,50.9901,36.8812,14.1089,36.8812,12.1287,0.600806,14439.6
1,Logistic Regression,52.7228,5.94059,46.7822,4.20792,43.0693,0.200837,1124.65
2,"Buy First Day, Sell Last Day",None,None,None,None,None,None,22317.3


I performed additional analysis, documented in the table below, where I varied included columns and Scaling. The analysis proved illuminating and shows how these different variables can affect the accuracy of the results.

|   __Incl. both Price and Timestamp - Use Scaler__                             |                       |                       |                     |                      |                      |                      |                 |                 |
|-------------------------------------------------------------------------------|-----------------------|-----------------------|---------------------|----------------------|----------------------|----------------------|-----------------|-----------------|
|                                                                               | Technique             | % Predicted Correctly | % of True Positives | % of True Negatives  | % of False Positives | % of False Negatives | F1 Score        | Final Net Worth |
| 0                                                                             | KNN                   | 49.2574               | 35.6436             | 13.6139              | 37.3762              | 13.3663              | 0.584178        | 6456.47         |
| 1                                                                             | LR                    | 48.5149               | 18.3168             | 30.198               | 20.7921              | 30.6931              | 0.41573         | 2250.7          |
| 2                                                                             | -                     | None                  | None                | None                 | None                 | None                 | None            | 22317.30        |
|   __Incl. both Price and Timestamp - No scaler (Converges to all T or all F)__|                       |                       |                     |                      |                      |                      |                 |                 |
|                                                                               | Technique             | % Predicted Correctly | % of True Positives | % of True Negatives  | % of False Positives | % of False Negatives | F1 Score        | Final Net Worth |
| 0                                                                             | KNN                   | 49.0099               | 49.0099             | 0                    | 50.9901              | 0                    | 0.657807        | 22317.3         |
| 1                                                                             | LR                    | 50.9901               | 0                   | 50.9901              | 0                    | 49.0099              | 0               | 1000            |
| 2                                                                             | -                     | None                  | None                | None                 | None                 | None                 | None            | 22317.3         |
|   __Incl. only price - Use Scaler__                                           |                       |                       |                     |                      |                      |                      |                 |                 |
|                                                                               | Technique             | % Predicted Correctly | % of True Positives | % of True Negatives  | % of False Positives | % of False Negatives | F1 Score        | Final Net Worth |
| 0                                                                             | KNN                   | 50.9901               | 36.8812             | 14.1089              | 36.8812              | 12.1287              | 0.600806        | 14439.6         |
| 1                                                                             | LR                    | 52.7228               | 5.94059             | 46.7822              | 4.20792              | 43.0693              | 0.200837        | 1124.65         |
| 2                                                                             | -                     | None                  | None                | None                 | None                 | None                 | None            | 22317.3         |
|   __Incl. only price - No Scaler__                                            |                       |                       |                     |                      |                      |                      |                 |                 |
|                                                                               | Technique             | % Predicted Correctly | % of True Positives | % of True Negatives  | % of False Positives | % of False Negatives | F1 Score        | Final Net Worth |
| 0                                                                             | KNN                   | 52.2277               | 39.604              | 12.6238              | 38.3663              | 9.40594              | 0.623782        | 18298.6         |
| 1                                                                             | LR                    | 51.4851               | 2.47525             | 49.0099              | 1.9802               | 46.5347              | 0.0925926       | 1437.89         |
| 2                                                                             | -                     | None                  | None                | None                 | None                 | None                 | None            | 22317.3         |
|   __Remove both Price and Timestamp - Use Scaler__                            |                       |                       |                     |                      |                      |                      |                 |                 |
| Technique                                                                     | % Predicted Correctly | % of True Positives   | % of True Negatives | % of False Positives | % of False Negatives | F1 Score             | Final Net Worth |                 |
| 0                                                                             | KNN                   | 47.5248               | 24.0099             | 23.5149              | 27.4752              | 25                   | 0.477833        | 4966.28         |
| 1                                                                             | LR                    | 55.6931               | 22.7723             | 32.9208              | 18.0693              | 26.2376              | 0.506887        | 4827.51         |
| 2                                                                             | -                     | None                  | None                | None                 | None                 | None                 | None            | 22317.3         |
|   __Remove both Price and Timestamp - No Scaler__                             |                       |                       |                     |                      |                      |                      |                 |                 |
|                                                                               | Technique             | % Predicted Correctly | % of True Positives | % of True Negatives  | % of False Positives | % of False Negatives | F1 Score        | Final Net Worth |
| 0                                                                             | KNN                   | 47.7723               | 24.2574             | 23.5149              | 27.4752              | 24.7525              | 0.481572        | 5817.13         |
| 1                                                                             | LR                    | 55.4455               | 17.8218             | 37.6238              | 13.3663              | 31.1881              | 0.444444        | 3115.27         |
| 2                                                                             | -                     | None                  | None                | None                 | None                 | None                 | None            | 22317.3         |

As you can see, none of the predictive approaches were more effective than buying on the first day and selling on the last day of the training set, though that may be in part due to the nature of the dataset, where the values generally rose from beginning to end.

That being said, some results were more effective than others. Including the UnixTimeStamp field yielded the worst result, presumably because it is time-series data that skews the predictive algorithms. Including price actually provided the best results for KNN, but some of the worst results for Logistic Regression.

Additional analysis would include manually varying the number of neighbors in KNN and improving upon the buy/sell algorithm to more effectively increase earnings